<a href="https://colab.research.google.com/github/priyanshsaxena24/Deep-Learning-UEC642--Project/blob/main/Handwriting_Ensemble_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

print("TensorFlow version:", tf.__version__)
print("GPUs:", tf.config.list_physical_devices("GPU"))
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train.astype("float32") / 255.0
x_test  = x_test.astype("float32") / 255.0
x_train = x_train[..., tf.newaxis]
x_test  = x_test[..., tf.newaxis]

num_classes = 10
y_train_oh = tf.keras.utils.to_categorical(y_train, num_classes=num_classes)
y_test_oh  = tf.keras.utils.to_categorical(y_test, num_classes=num_classes)

from tensorflow.keras.layers import SeparableConv2D

def build_separable_cnn(input_shape=(28, 28, 1), num_classes=10):
    model = models.Sequential([
        SeparableConv2D(32, (3, 3), padding="same", activation="relu",
                        input_shape=input_shape),
        SeparableConv2D(32, (3, 3), activation="relu"),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),

        SeparableConv2D(64, (3, 3), padding="same", activation="relu"),
        SeparableConv2D(64, (3, 3), activation="relu"),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),

        layers.Flatten(),
        layers.Dense(128, activation="relu"),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax")
    ])
    return model

model = build_separable_cnn()
model.summary()

loss_fn = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss=loss_fn,
    metrics=["accuracy"]
)


models_list = []
num_models = 3

for i in range(num_models):
    print(f"\nTraining model {i+1}/{num_models}")
    m = build_separable_cnn()
    m.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss=loss_fn,
        metrics=["accuracy"]
    )
    m.fit(
        x_train, y_train_oh,
        batch_size=128,
        epochs=20,
        validation_split=0.1,
        callbacks=[early_stop, reduce_lr],
        verbose=2
    )
    models_list.append(m)

import numpy as np

probs = [m.predict(x_test, verbose=0) for m in models_list]
probs_mean = np.mean(probs, axis=0)
y_pred = np.argmax(probs_mean, axis=1)
ensemble_acc = np.mean(y_pred == y_test)

print(f"Single best model acc ≈ {max(m.evaluate(x_test, y_test_oh, verbose=0)[1] for m in models_list):.4f}")
print(f"Ensemble accuracy: {ensemble_acc:.4f}")

early_stop = EarlyStopping(
    monitor="val_accuracy",
    patience=3,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=2,
    min_lr=1e-5
)

history = model.fit(
    x_train, y_train_oh,
    batch_size=128,
    epochs=25,
    validation_split=0.1,
    callbacks=[early_stop, reduce_lr],
    verbose=2
)

test_loss, test_acc = model.evaluate(x_test, y_test_oh, verbose=0)
print(f"Test accuracy with separable conv + label smoothing: {test_acc:.4f}")


TensorFlow version: 2.19.0
GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ separable_conv2d_4              │ (None, 28, 28, 32)     │            73 │
│ (SeparableConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ separable_conv2d_5              │ (None, 26, 26, 32)     │         1,344 │
│ (SeparableConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ separable_conv2d_6              │ (None, 13, 13, 64)     │         2,400 │
│ (SeparableConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ separable_conv2d_7              │ (None, 11, 11, 64)     │         4,736 │
│ (SeparableConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 214,771 (838.95 KB)

 Trainable params: 214,771 (838.95 KB)

 Non-trainable params: 0 (0.00 B)


Training model 1/3
Epoch 1/20
422/422 - 15s - 36ms/step - accuracy: 0.8026 - loss: 1.0518 - val_accuracy: 0.9697 - val_loss: 0.6356 - learning_rate: 1.0000e-03
Epoch 2/20
422/422 - 3s - 6ms/step - accuracy: 0.9529 - loss: 0.7191 - val_accuracy: 0.9817 - val_loss: 0.5930 - learning_rate: 1.0000e-03
Epoch 3/20
422/422 - 3s - 6ms/step - accuracy: 0.9664 - loss: 0.6804 - val_accuracy: 0.9863 - val_loss: 0.5747 - learning_rate: 1.0000e-03
Epoch 4/20
422/422 - 3s - 7ms/step - accuracy: 0.9713 - loss: 0.6611 - val_accuracy: 0.9885 - val_loss: 0.5657 - learning_rate: 1.0000e-03
Epoch 5/20
422/422 - 3s - 6ms/step - accuracy: 0.9761 - loss: 0.6460 - val_accuracy: 0.9892 - val_loss: 0.5592 - learning_rate: 1.0000e-03
Epoch 6/20
422/422 - 3s - 6ms/step - accuracy: 0.9784 - loss: 0.6368 - val_accuracy: 0.9898 - val_loss: 0.5531 - learning_rate: 1.0000e-03
Epoch 7/20
422/422 - 3s - 6ms/step - accuracy: 0.9799 - loss: 0.6301 - val_accuracy: 0.9905 - val_loss: 0.5535 - learning_rate: 1.0000e-03
Epoch